In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-notebook")
import seaborn as sns
from data.cleaning import merge_datetime, get_dataset_from_pickle

# City of Los Angeles: Proposing a Strategy for Optimizing Parking Enforcement Deployment
**Author: Evan Gabrielson**

### Overview: 
---
In recent years, the City of Los Angeles has faced a significant financial challenge with its parking enforcement operations. Although traffic fines once provided a steady stream of revenue, a troubling shift occurred starting in 2017. Since then, the costs associated with salaries, equipment, and other expenses for parking enforcement have skyrocketed to over $809 million, while the revenue generated from parking ticket fines has lagged behind at $617 million. This $192 million shortfall highlights the urgent need for more efficient and effective strategies in managing parking violations and enforcement.

### Business Understanding:
---
In his book titled "The High Cost of Free Parking", LA resident and UCLA professor Donald Shoup outlines the necessity of parking enforcement policy to "curb" the excess time and resources a free parking state begets. 
Shoup presents two key recommendations for reform to improve parking policy: pricing curb parking according to fair market value and redistributing parking revenue to neighborhoods for community investment.
Parking enforcement is only necessary to the extent that it improves the maintenance and fair distribution of parking resources such that all actors can benefit equally from public road infrastructure.
Today, however, LA Department of Transportation (LADOT) policy makers are forced to counteract skyhigh salary expenses with parking violation fees well above fair market value while reinvesting nothing back into the communities. 
Until LADOT can produce a net profit from parking enforcement, the citizenry of Los Angeles must continue to expect rising parking violation fees and zero community reinvestment.

In this project, I propose several data-driven strategies for optimizing parking enforcer deployment which LADOT can employ to close the gap between revenue and payroll. 
Here are some facts about LADOT as it functions today:
- LADOT currently deploys an equal distribution of enforcement officers across the City of LA. 
- Enforcers are on duty 24/7
- Parking citations fees range from $53 - $350

### Data:
---
Parking citation data is available through the [City of LA's Data Catalog](https://data.lacity.org/Transportation/Parking-Citations/4f5p-udkv/about_data) and is updated on a daily basis. The following columns are of interest:
- `issue_date`
- `issue_time`
- `fine_amount`
- `agency`
- `violation_code`
- `loc_lat`
- `loc_long`

#### Data preparation and cleaning
The `utils.py` script converts the dataset retrieved in JSON format from the Data Catalog API to a pandas DataFrame, ignoring unnecessary columns and invalid rows before storing the cleaned DataFrame to a pickle file. By "invalid rows" I refer to rows with missing values in critical columns such as `issue_date`, `issue_time` or `violation_code`.

In [ ]:
df = get_dataset_from_pickle("data/pickle/citations_v0.pickle")
df.info()

#### Creating a DateTimeIndex

We have two columns `issue_date` and `issue_time` that need to be combined in order to use pandas' built-in DateTimeIndex helper functions and cut down on memory usage.

In [ ]:
# Create new datetime column `issue_datetime` from the combined `issue_date` and `issue_time` columns
df['issue_datetime'] = merge_datetime(df, 'issue_date', 'issue_time')

# Drop unused columns
df = df.drop(columns=['issue_date', 'issue_time'])

# Set datetime index
df = df.set_index('issue_datetime')

# Only use violation data for 2014 through 2023
df = df.loc[(df.index > '2013-12-31') & (df.index < '2024-1-1')]

df.info()

#### Casting Datatypes

Casting datatypes can help reduce the size of the DataFrame while minimizing the amount of error handling required later on in our analysis. The location of each citation is stored as longitude (`loc_long`) and latitude (`loc_lat`) values, which should be cast to floating-point columns. `fine_amount` should similarly be a floating-point column. We'll use `np.float32` to store these values as 32-bit floats.

In [ ]:
# Cast numerical columns to correct datatype
df['loc_lat'] = df['loc_lat'].astype('float32')
df['loc_long'] = df['loc_long'].astype('float32')
df['fine_amount'] = df['fine_amount'].astype('float32')

df.info()

Two features remain as generic `object` type columns: `agency` and `violation_code`. Let's quickly identify if either of these are categorical columns by checking the number of unique values in each column using the `describe()` DataFrame member function. Categorical features are helpful for encoding string-type values into numerical values for machine learning algorithms and also for analyzing groups of rows.

In [ ]:
df[['agency', 'violation_code']].describe()

In [ ]:
df['agency'] = df['agency'].astype('category')
df['violation_code'] = df['violation_code'].astype('category')

df.info()

#### Missing & Invalid Data

There are several columns containing rows with null-ish values (i.e. NaN or 0.0). There are 65,080 rows with missing values which is likely due to data entry errors and optional fields on the citation form. For the sake of this analysis, we will ignore these rows which only occupy 0.3% of the total dataset and appear to be randomly distributed across the feature set.

In [2]:
df = get_dataset_from_pickle('data/pickle/citations_v0_bronze.pickle')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20594138 entries, 2022-12-13 16:20:58.225000 to 2023-12-27 10:07:08.455000
Data columns (total 5 columns):
 #   Column          Dtype   
---  ------          -----   
 0   fine_amount     float32 
 1   agency          category
 2   violation_code  category
 3   loc_lat         float32 
 4   loc_long        float32 
dtypes: category(2), float32(3)
memory usage: 451.7 MB


In [3]:
# Isolate rows with columns containing NaN or 0.0 indicating an invalid value
nullish_rows = (df['loc_lat'].isna() | (df['loc_lat'] == 0.0)) | (df['loc_long'].isna() | (df['loc_long'] == 0.0)) | (df['fine_amount'].isna() | (df['fine_amount'] == 0.0)) | (df['agency'].isna())
print(f"There are {nullish_rows.sum()} with invalid or null values.")

# Drop nullish rows
df = df[nullish_rows == False]

There are 65080 with invalid or null values.


In [4]:
df.isnull().sum()

fine_amount       0
agency            0
violation_code    0
loc_lat           0
loc_long          0
dtype: int64

In [5]:
(df[['loc_lat', 'loc_long', 'fine_amount']] == 0).sum()

loc_lat        0
loc_long       0
fine_amount    0
dtype: int64

**CHECKPOINT REACHED**

We've now succesfully shrunk the DataFrame size by 65% from 1.3+ GB to 451 MB while standardizing the datetime, numerical, and categorical datatypes across all of the columns. Using the [Medallion data design pattern](https://learn.microsoft.com/en-us/azure/databricks/lakehouse/medallion), we'll store progressively higher quality DataFrames into data stores labeled "Bronze", then "Silver", then "Gold". At this data processing checkpoint, we'll store the current DataFrame as a "Bronze"-tier pickle file.

In [6]:
df.to_pickle('data/pickle/citations_v0_bronze.pickle')
df.head()

,fine_amount,agency,violation_code,loc_lat,loc_long
issue_datetime,,,,,
2022-12-13 16:20:58.225,50.0,1,4000A1,34.234493,-118.436005
2022-12-13 16:25:41.006,50.0,1,4000A1,34.234493,-118.436005
2022-12-16 12:10:48.627,50.0,1,4000A1,34.242790,-118.450142
2022-12-19 08:45:06.121,50.0,1,4000A1,34.243561,-118.450386
2022-12-19 08:55:35.605,50.0,1,4000A1,34.243561,-118.450386


Navigate to `01_exploratory_data_analysis.ipynb` to continue with the project walkthrough